In [ ]:
import os, random, shutil
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

base_dir
  1- train_dir  
    a- train_neg_dir
    b- train_pos_dir
  2- val_dir
    a- val_neg_dir
    b- val_pos_dir
  3- test_dir
    a- test_neg_dir
    b- test_pos_dir

dataset_dir is seperated

In [ ]:
class hpTrain():
    def divid_img(self, dset_dir='', base_dir='', ratio=.2, pref=''):
        self.dataset_dir = dset_dir
        os.makedirs(base_dir, exist_ok=True)
        self.train_dir = os.path.join(base_dir, 'train')
        os.makedirs(self.train_dir, exist_ok=True)
        self.validation_dir = os.path.join(base_dir, 'validation')
        os.makedirs(self.validation_dir, exist_ok=True)
        self.test_dir = os.path.join(base_dir, 'test')
        os.makedirs(self.test_dir, exist_ok=True)
        self.train_negative_dir = os.path.join(self.train_dir, 'negative')
        os.makedirs(self.train_negative_dir, exist_ok=True)
        self.train_positive_dir = os.path.join(self.train_dir, 'positive')
        os.makedirs(self.train_positive_dir, exist_ok=True)
        self.validation_negative_dir = os.path.join(self.validation_dir, 'negative')
        os.makedirs(self.validation_negative_dir, exist_ok=True)
        self.validation_positive_dir = os.path.join(self.validation_dir, 'positive')
        os.makedirs(self.validation_positive_dir, exist_ok=True)
        self.test_negative_dir = os.path.join(self.test_dir, 'negative')
        os.makedirs(self.test_negative_dir, exist_ok=True)
        self.test_positive_dir = os.path.join(self.test_dir, 'positive')
        os.makedirs(self.test_positive_dir, exist_ok=True)
        if pref == 'negative':
            train_cls_dir = self.train_negative_dir
            validation_cls_dir = self.validation_negative_dir
            test_cls_dir = self.test_negative_dir
        else:
            train_cls_dir = self.train_positive_dir
            validation_cls_dir = self.validation_positive_dir
            test_cls_dir = self.test_positive_dir
        imgs = [i for i in os.listdir(self.dataset_dir) if i.startswith(pref)]
        cnt = int(len(imgs) * ratio)
        random.shuffle(imgs)
        for fname in imgs[:cnt]:
            src = os.path.join(self.dataset_dir, fname)
            dst = os.path.join(validation_cls_dir, fname)
            shutil.move(src, dst)
        print('total %s validation images: %d' % (pref, len(os.listdir(validation_cls_dir))))
        imgs = [i for i in os.listdir(self.dataset_dir) if i.startswith(pref)]
        random.shuffle(imgs)
        for fname in imgs[:cnt]:
            src = os.path.join(self.dataset_dir, fname)
            dst = os.path.join(test_cls_dir, fname)
            shutil.move(src, dst)
        print('total %s test images: %d' % (pref, len(os.listdir(test_cls_dir))))
        imgs = [i for i in os.listdir(self.dataset_dir) if i.startswith(pref)]
        random.shuffle(imgs)
        for fname in imgs:
            src = os.path.join(self.dataset_dir, fname)
            dst = os.path.join(train_cls_dir, fname)
            shutil.move(src, dst)
        print('total %s train images: %d' % (pref, len(os.listdir(train_cls_dir))))

    def train(self, bsize=16, epo=100):
        # def model
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu',
                                input_shape=(255, 255, 3)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        # compile model
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizers.RMSprop(),
                      metrics=['accuracy'])
        train_datagen = ImageDataGenerator(
            rescale=1. / 255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1. / 255)
        train_generator = train_datagen.flow_from_directory(self.train_dir,
            target_size=(255, 255),
            batch_size=bsize,
            class_mode='binary')
        validation_generator = test_datagen.flow_from_directory(self.validation_dir,
            target_size=(255, 255),
            batch_size=bsize,
            class_mode='binary')

        callbacks = [EarlyStopping(patience=10, verbose=1),
                    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
                    ModelCheckpoint('hp.h5', verbose=1, save_best_only=True, save_weights_only=True)]

        model.fit(
            train_generator,
            steps_per_epoch=len(os.listdir(self.train_negative_dir)) * 2 // bsize,
            epochs=epo,
            validation_data=validation_generator,
            validation_steps=len(os.listdir(self.validation_negative_dir)) * 2 // bsize,
            callbacks=callbacks)
        # evaluate test
        test_generator = test_datagen.flow_from_directory(self.test_dir,
                                                          target_size=(255, 255),
                                                          batch_size=bsize,
                                                          class_mode='binary')
        test_loss, test_acc = model.evaluate(test_generator, steps=len(os.listdir(self.test_negative_dir)) * 2 // bsize)
        print('Test accuracy: ', test_acc)
        test_generator.reset()
        preds = model.predict(validation_generator, verbose=1)
        fpr, tpr, _ = roc_curve(validation_generator.classes, preds)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        lw = 2
        plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
def main():
    prefs = ['negative', 'positive']
    hp = hpTrain()
    for i in prefs:
        hp.divid_img(dset_dir="C:\\New folder\\hp project\\dogs", base_dir="C:\\New folder\\hp project\\base_dir", pref=i)
    hp.train(epo=1)


if '__main__' == __name__:
    main()